# 🦌 Compounds 🧪

Experiments on building graph fragments that can be composed. This is tricky because to
make reusable fragments a new `id` will have to be stamped out for each element. This
notebook introduces the following elements in `ipyelk.contrib.elements`:

- `Node` - wrapper for `ElkNode`
- `Port` - wrapper for `ElkPort`
- `Label` - wrapper for `ElkLabel`
- `Edge` - wrapper for `ElkEdge`
- `Partition` - extends node and has some convience functions for building edges

To stamp out `id`s while remembering the originating objects, a `Compound` class can be
instantiated that owns the `Registry`.

In [ ]:
import importnb
import ipywidgets as W
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.elements import Compound, Edge, Label, Node, Port

# from ipyelk.contrib.elements.base import Element
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.symbol import ConnectorDef, Def, Symbol, symbols

`ipyelk.contrib.library.activity` extends the base `Elements` into a set of marks that
are appropriate for creating Activity Diagrams. These new marks do not have behaviors or
rules that enforce for how they can be connected.

In [ ]:
from ipyelk.contrib.library.activity import (
    Activity,
    ActivityDiagram,
    Decision,
    EndActivity,
    Join,
    Merge,
    StartActivity,
)


def activity_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="DOWN"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app

## Example Email Activities

Simple representation of processing an email inbox.

In [ ]:
def email_activity_example():

    # Building Elements
    act = ActivityDiagram()
    root = act.partition

    start = StartActivity()
    end = EndActivity()

    open_email = Activity.make("open email")
    delete_email = Activity.make("delete email")
    read_email = Activity.make("read email")
    reply_email = Activity.make("reply")

    j1 = Join()

    m1 = Merge()

    triage = Decision()
    triage.true.labels = [Label(text="is important")]
    triage.false.labels = [Label(text="is junk")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connect Elements
    root[start:open_email]
    root[open_email : triage.input : "opening"]
    root[triage.false : delete_email]
    root[delete_email:m1]
    root[triage.true : read_email]
    root[read_email : response.input]
    root[response.false : m1]
    root[response.true : reply_email]
    root[reply_email:m1]
    root[m1:end]

    # build app
    ilk = Compound()
    app = activity_app()
    app.transformer.source = ilk(root)
    app.diagram.defs = act.defs
    app.style = act.style
    return app, act

In [ ]:
if __name__ == "__main__":
    email_act_app, email_activities = email_activity_example()
    display(email_act_app)

## Example Email Activities

Simple representation of processing an email inbox.

In [ ]:
def website_activity_example():
    priority_edge_opts = {
        "org.eclipse.elk.layered.priority.direction": "10",
    }

    # Building Elements
    act = ActivityDiagram()
    root = act.partition

    start = StartActivity()
    end = EndActivity()

    landing = Activity.make("Landing Page")
    login = Activity.make("Login", container=True)
    enter_creds = Activity.make("Enter Credentials")
    register = Activity.make("Register", container=True)
    registration = Activity.make("Enter Registration Data")
    confirm_email = Activity.make("Receive Confirmation Email")
    confirm = Activity.make("Click Confirmation Link")

    website = Activity.make("Explore Website")

    login.add_child(enter_creds)

    for c in [registration, confirm_email, confirm]:
        register.add_child(c)

    d1 = Decision()
    d1.true.labels = [Label(text="registered")]
    d1.false.labels = [Label(text="not registered")]

    d2 = Decision()
    d2.true.labels = [Label(text="logged in")]
    d2.false.labels = [Label(text="not logged in")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connecting Elements
    root[start:landing].layoutOptions.update(priority_edge_opts)
    root[landing : d1.input].layoutOptions.update(priority_edge_opts)
    root[d1.true : enter_creds]
    root[d1.false : registration]
    root[registration:confirm_email]
    root[confirm_email:confirm]

    m1 = Merge()
    root[enter_creds:m1]
    root[confirm:m1]
    root[m1 : d2.input]
    root[d2.false : landing]
    root[d2.true : website]

    root[website:end]

    # Creating App and setting the source
    ilk = Compound()
    app = activity_app()
    app.transformer.source = ilk(root, register, login)
    app.diagram.defs = act.defs
    app.style = act.style
    return app

In [ ]:
if __name__ == "__main__":
    website_app = website_activity_example()
    display(website_app)

# Record Nodes

In [ ]:
import importnb
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.elements import (
    Compartment,
    Compound,
    Edge,
    Label,
    Mark,
    Node,
    Port,
    Record,
)
from ipyelk.contrib.library.block import Aggregation, Block, BlockDiagram, Composition

# from ipyelk.contrib.elements.base import Element
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram.symbol import ConnectorDef, Def, Symbol, symbols
from ipyelk.tools import ToolButton


class ToggleCollapsedBtn(ToolButton):
    @T.default("description")
    def _default_description(self):
        return "Toggle Collapsed"

    def toggle(self, node):
        """Toggle the `hidden` state for the given networkx node"""
        tree = self.app.transformer.source[1]
        state = tree.nodes[node].get("hidden", False)
        tree.nodes[node]["hidden"] = not state

    def handler(self, *args):
        should_refresh = False
        for selected in self.app.selected:
            for node in self.get_related(selected):
                self.toggle(node)
                should_refresh = True

        # trigger refresh if needed
        if should_refresh:
            self.app.refresh()

    def get_related(self, node):
        tree = self.app.transformer.source[1]
        if tree and node in tree:
            return tree.neighbors(node)
        return []


class ToggleRecordBtn(ToggleCollapsedBtn):
    def get_related(self, node):
        tree = self.app.transformer.source[1]
        if isinstance(node, Mark) and isinstance(node.node, Compartment):
            parent = list(tree.predecessors(node))[0]
            for i, child in enumerate(tree.neighbors(parent)):
                if i > 0:  # still the first compartment in the record
                    yield child
        return super().get_related(node)


def block_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="RIGHT"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ToggleRecordBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app


app2 = block_app()
app2

In [ ]:
bd = BlockDiagram()

# Nodes
vehicle = Block()
vehicle.title = Compartment(headings=["Vehicle", "«block»"])

wheel = Block(width=180)
wheel.title = Compartment(headings=["Wheel", "«block»"])
wheel.attrs = Compartment(headings=["properties"], content=["- radius: float"])

# Edges
bd.partition[vehicle:wheel:Composition]

cp = Compound()
app2.transformer.source = cp(bd.partition)
app2.style = bd.style
app2.diagram.defs = bd.defs

In [ ]:
import networkx as nx


def compose(c1, c2):
    g = nx.compose(c1[0], c2[0])
    tree = nx.compose(c1[1], c2[1])
    return g, tree


bd = BlockDiagram()

b = Block()
b.width = 100
b.title = Compartment(headings=["Point", "«class»"])
b.attrs = Compartment(headings=["Attributes"], content=["- x: float", "- y: float"])

cp = Compound()
cp2 = Compound()

bd.partition

# app.diagram.defs = logic_gates.Gate.make_defs()
app2.transformer.source = compose(cp(b), cp2(b))
app2.style = bd.style

In [ ]:
g = compose(cp(b), cp2(b))[0]